# Passport Problem 🚴


Example modified from [here](https://jump.dev/JuMP.jl/stable/tutorials/getting_started/getting_started_with_data_and_plotting/#Example:-the-passport-problem).

[Passport Index Dataset](https://github.com/ilyankou/passport-index-dataset).

### What's the minimum number of passports that I need to visit all 199 countries in the world?

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f9/Physical_World_Map.svg/2880px-Physical_World_Map.svg.png)

In [35]:
# !wget https://raw.githubusercontent.com/ilyankou/passport-index-dataset/master/passport-index-matrix.csv -O data/passport-index-matrix.csv

In [36]:
using CSV
using DataFrames
using JuMP
using HiGHS

In [37]:
passport_data = CSV.read(
    joinpath("./data/", "passport-index-matrix.csv"),
    DataFrames.DataFrame,
)

Row,Passport,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,DR Congo,Costa Rica,Ivory Coast,Croatia,Cuba,Cyprus,Czech Republic,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Swaziland,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,⋯
,String,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String7,String7,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,⋯
1,Afghanistan,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,e-visa,visa required,visa required,e-visa,e-visa,visa on arrival,visa required,visa required,visa required,visa required,e-visa,e-visa,visa on arrival,visa required,e-visa,visa required,visa required,visa required,e-visa,visa on arrival,visa on arrival,e-visa,visa required,visa on arrival,visa required,visa required,visa required,visa required,e-visa,visa on arrival,visa required,e-visa,visa required,e-visa,visa required,visa required,visa required,visa required,visa required,e-visa,21,visa required,e-visa,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,e-visa,visa required,visa required,visa required,visa required,visa required,e-visa,visa on arrival,visa required,90,visa required,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,visa required,visa required,visa required,visa required,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,⋯
2,Albania,-1,visa required,90,visa required,180,visa required,180,e-visa,90,e-visa,e-visa,e-visa,visa on arrival,28,30,90,visa required,e-visa,e-visa,visa on arrival,90,e-visa,90,visa required,90,e-visa,visa on arrival,visa on arrival,e-visa,visa required,visa on arrival,visa required,visa required,90,90,90,visa on arrival,visa required,e-visa,visa required,e-visa,90,visa required,90,90,90,e-visa,21,visa free,e-visa,visa on arrival,90,e-visa,visa required,90,visa required,e-visa,visa required,90,90,e-visa,90,360,90,visa required,90,visa required,visa required,e-visa,visa on arrival,90,90,visa required,14,90,90,e-visa,visa on arrival,e-visa,visa required,visa required,90,90,visa on arrival,visa required,visa on arrival,90,e-visa,visa required,90,visa required,60,visa on arrival,90,visa required,e-visa,visa required,visa required,90,⋯
3,Algeria,e-visa,-1,visa required,30,e-visa,visa required,e-visa,e-visa,visa required,e-visa,e-visa,e-visa,visa required,90,visa required,visa required

In [38]:
# 1 if visa is not required, 0 if visa required 

function modifier(x)
    if x == -1 || (x != "visa required" && x != "e-visa" && x != "visa on arrival")
        return 1
    else
        return 0
    end
end

for country in passport_data.Passport
    passport_data[!, country] = modifier.(passport_data[!, country])
end

In [39]:
passport_data

Row,Passport,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,DR Congo,Costa Rica,Ivory Coast,Croatia,Cuba,Cyprus,Czech Republic,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Swaziland,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,⋯
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,Albania,1,0,1,0,1,0,1,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,1,1,0,1,1,0,0,1,0,0,1,0,0,0,1,1,0,1,1,1,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,⋯
3,Algeria,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,Andorra,1,0,1,0,1,1,1,0,1,0,1,0,0,1,1,1,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,1,1,0,1,1,1,0,1,0,1,0,0,0,1,1,1,1,1,0,0,0,0,1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,⋯
5,Angola,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,Antigua and Barbuda,1,0,1,1,1,0,0,0,1,0,1,0,0,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,1,1,1,1,0,1,1,1,0,1,0,0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,⋯
7,Argentina,1,0,1,1,1,1,1,0,1,0,1,0,0,1,1,1,1,0,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,1,1,1,1,0,1,1,0,1,1,1,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,⋯
8,Armenia,1,0,0,0,1,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,⋯
9,Australia,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,1,1,0,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,1,0,1,1,0,1,0,1,0,1,0,0,1,⋯


In [40]:
# List of countries
C = passport_data.Passport;

In [41]:
model = Model(HiGHS.Optimizer)
set_silent(model)

## Control variable 

For every country, we define a binary variable $p_{c} \in \{0,1\}$. 

* $p_{c} = 1$ would indicate that **we have** a passport for that country
* $p_{c} = 0$ would indicate that **we don't have** a passport for that country

| | | | | | ||
|--|--|--|--|--|--|--|
Country | 🇨🇴 |🇦🇷|🇱🇹 |🇨🇭|🇬🇷 |...
$p_c$ |? |? | ?|? |? |  ... 

In [42]:

@variable(model, p[C], Bin);

## Objective function 

And we want to minimize the number of passports we need to have

$$
\begin{align*}
\min_{p_{c}} & &\sum p_c\\
\end{align*}
$$

In [43]:
@objective(model, Min, sum(p));


## Constraints 

Since we wish to visit all the countries, for every country, we must own at least one passport that lets us travel to that country visa free.

For a given country `X`, the vector `passport_data[!, X]` has 199 entries (one for each country), indicating wheather a visa is required for each of the countries to enter `X`. 

* 0 if visa is required
* 1 if visa is not required

For instance, to enter the States 🇺🇸

| | | | | | ||
|--|--|--|--|--|--|--|
Country of origin | 🇺🇸 | 🇨🇦| 🇲🇽|🇨🇳 |🇵🇼 |...
Visa required? `passport_data[!,🇺🇸]` |1| 1|0 |0 | 1| ...
Do I have a passport? $p_c$ | ?|? | ?| ?| ?|...
`passport_data[!,🇺🇸]` $\cdot$ $p_c$ | | | | | |  ...


The constraint then consist of ensuring that for a given country we have at least one passport.

In [44]:
@constraint(model, [d in C], passport_data[!, d]' * p >= 1);

In [45]:
optimize!(model)

In [46]:
solution_summary(model)


* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 1.90000e+01
  Objective bound    : 1.90000e+01
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 4.55063e-03
  Simplex iterations : 25
  Barrier iterations : -1
  Node count         : 1


In [47]:
objective_value(model)

19.0

In [48]:
println("Optimal passports:")
for c in C
    if value(p[c]) > 0.5
        println(" * $c")
    end
end


Optimal passports:
 * Afghanistan
 * Australia
 * Bangladesh
 * China
 * Comoros
 * Congo
 * Israel
 * Kenya
 * Madagascar
 * Mali
 * Nepal
 * North Korea
 * Palau
 * Papua New Guinea
 * Romania
 * Singapore
 * Somalia
 * Turkmenistan
 * United Arab Emirates


In [49]:
# a = passport_data[!, "United States"]

# C[a .== 1]